In [1]:
import os
cwd = os.getcwd()

In [2]:
#vietnamese PhoW2V
#!gdown --id 1--bdQosASIlc3O4xN5IZ5vLfZ_yEdfBZ
try:
    f = open(cwd+"/vietnamese_PhoW2V_100dims.pickle")
    # Do something with the file
    f.close()
except IOError:
    !gdown --id 1--bdQosASIlc3O4xN5IZ5vLfZ_yEdfBZ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1--bdQosASIlc3O4xN5IZ5vLfZ_yEdfBZ
To: /content/vietnamese_PhoW2V_100dims.pickle
100% 1.32G/1.32G [00:07<00:00, 169MB/s]


In [3]:
import pickle
PhoW2V = pickle.load(open(cwd+'/vietnamese_PhoW2V_100dims.pickle','rb'))

In [4]:
import numpy as np

In [5]:
#PhoW2V = [words, word2idx, vectors]
words = PhoW2V[0]
#word2idx = PhoW2V_300dims[1]
vectors = PhoW2V[2]
del PhoW2V

In [6]:
words = ['<SOS>']+['<EOS>']+words

In [7]:
# vectors = np.concatenate(([np.zeros(300,)],vectors))
# vectors = np.concatenate(([np.zeros(300,)],vectors))

In [8]:
vectors = np.insert(vectors, 0, np.random.rand(100,)/10,axis=0)
vectors = np.insert(vectors, 0, np.random.rand(100,)/10,axis=0)

In [9]:
idx2word = {idx: words[idx] for idx in range(len(words))}
word2idx = {v: k for k, v in idx2word.items()}

In [10]:
vietnamese_w2v = {w: vectors[word2idx[w]] for w in words}

In [11]:
#dataset
try:
    f = open(cwd+"/ViMs.zip")
    # Do something with the file
    f.close()
except IOError:
    !gdown --id 1Fj-XA_QvJKMrk_jluEv08Nvp5sAFQn8F

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Fj-XA_QvJKMrk_jluEv08Nvp5sAFQn8F
To: /content/ViMs.zip
100% 7.01M/7.01M [00:00<00:00, 23.6MB/s]


In [12]:
!gdown --id 1Fj-XA_QvJKMrk_jluEv08Nvp5sAFQn8F

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Fj-XA_QvJKMrk_jluEv08Nvp5sAFQn8F
To: /content/ViMs.zip
100% 7.01M/7.01M [00:00<00:00, 276MB/s]


In [13]:
import zipfile
with zipfile.ZipFile(cwd+'/ViMs.zip', 'r') as zip_ref:
    zip_ref.extractall(cwd)

In [14]:
import glob

In [15]:
folder_path = '/content/ViMs/original'
texts = []
for filename in glob.glob(os.path.join(folder_path,'*','*','*.txt')):
    with open(filename) as f:
      text = f.readlines()
      texts.append(text)

In [16]:
titles = []
summaries = []
contents = []
for text in texts:
    titles.append(text[0][7:])
    summaries.append(text[6][9:])
    content = text[8:]
    content = ' '.join(content)
    content = content.replace('\n','')
    contents.append(content)

In [17]:
!pip3 install py_vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
  Created wheel for py-vncorenlp: filename=py_vncorenlp-0.1.3-py3-none-any.whl size=4309 sha256=8899ace4870a1de9bf6e7a8a0c0c54a03a81d76d40876ed6042fa1a67aeead23
  Stored in directory: /root/.cache/pip/wheels/87/be/55/d5930c1d90a09832e9afd57a9e13801e3d16c88f5a19f777ae
Successfully built py-vncorenlp


In [18]:
import py_vncorenlp
import os
cwd = os.getcwd()
py_vncorenlp.download_model(save_dir=cwd)
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=cwd)

In [19]:
# for list of sentences
segmented_titles = [' '.join(rdrsegmenter.word_segment(title)) for title in titles]
segmented_summaries = [' '.join(rdrsegmenter.word_segment(summary)) for summary in summaries]
segmented_contents = [' '.join(rdrsegmenter.word_segment(content)) for content in contents]

In [20]:
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
# tokenizing
segmented_titles = [['<SOS>']+tokenize.word_tokenize(title)+['<EOS>'] for title in segmented_titles]
segmented_summaries = [['<SOS>']+tokenize.word_tokenize(sum)+['<EOS>'] for sum in segmented_summaries]
segmented_contents = [['<SOS>']+tokenize.word_tokenize(content)+['<EOS>'] for content in segmented_contents]

In [22]:
from unicodedata import normalize

In [23]:
def word2idx_edit(word):
    try:
        return word2idx[normalize('NFC', word)]
    except:
        return -1

In [24]:
# word2ids
segmented_titles = [[word2idx_edit(normalize('NFC', word)) for word in title] for title in segmented_titles]
segmented_summaries = [[word2idx_edit(normalize('NFC', word)) for word in sum] for sum in segmented_summaries]
segmented_contents = [[word2idx_edit(normalize('NFC', word)) for word in content] for content in segmented_contents]

In [25]:
for i in segmented_titles:
    while -1 in i:
      i.remove(-1)
for i in segmented_summaries:
    while -1 in i:
      i.remove(-1)
for i in segmented_contents:
    while -1 in i:
      i.remove(-1)

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [27]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
pairs = [(torch.tensor(segmented_summaries[i]).long().to(device),torch.tensor(segmented_titles[i]).long().to(device)) for i in range(len(segmented_titles))]

In [29]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [30]:
MAX_LENGTH = 117
#2623

In [31]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [32]:
import random

In [33]:
teacher_forcing_ratio = 0.5
SOS_token = 0
EOS_token = 1

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        try:
            encoder_outputs[ei] = encoder_output[0, 0]
        except:
            print(encoder_outputs)
            print(len(encoder_outputs))
            print(ei)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output[0], target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output[0], target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    # for di in range(target_length):
    #     decoder_output, decoder_hidden, decoder_attention = decoder(
    #         decoder_input, decoder_hidden, encoder_outputs)
    #     topv, topi = decoder_output.topk(1)
    #     decoder_input = topi.squeeze().detach()  # detach from history as input
    #     # print(decoder_output)
    #     # print(target_tensor)
    #     loss += criterion(decoder_output[0], target_tensor[di])
    #     if decoder_input.item() == EOS_token:
    #         break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [34]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [35]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [36]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [random.choice(pairs) for i in range(n_iters)]
    #np.random.choice(pairs, n_iters) [random.choice(pairs) for i in range(n_iters)]
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [37]:
def evaluate(encoder, decoder, input_tensor, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [38]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [39]:
hidden_size = 64
num_vocab = len(words)
encoder1 = EncoderRNN(num_vocab, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, num_vocab, dropout_p=0.1).to(device)

In [40]:
trainIters(encoder1, attn_decoder1, n_iters=10000, print_every=500)

3m 56s (- 74m 49s) (500 5%) 13.7220
7m 28s (- 67m 13s) (1000 10%) 9.8365
10m 40s (- 60m 29s) (1500 15%) 7.1487
13m 52s (- 55m 30s) (2000 20%) 6.8833
17m 0s (- 51m 1s) (2500 25%) 6.6406
20m 14s (- 47m 13s) (3000 30%) 7.0226
23m 37s (- 43m 52s) (3500 35%) 7.5354
26m 57s (- 40m 26s) (4000 40%) 7.4859
30m 13s (- 36m 56s) (4500 45%) 7.0492
33m 37s (- 33m 37s) (5000 50%) 7.4374
37m 3s (- 30m 19s) (5500 55%) 7.6390
40m 30s (- 27m 0s) (6000 60%) 7.7546
43m 59s (- 23m 41s) (6500 65%) 7.8048
47m 27s (- 20m 20s) (7000 70%) 7.8186
50m 59s (- 16m 59s) (7500 75%) 8.0219
54m 35s (- 13m 38s) (8000 80%) 7.7065
58m 4s (- 10m 14s) (8500 85%) 7.5958
61m 40s (- 6m 51s) (9000 90%) 7.5706
65m 13s (- 3m 25s) (9500 95%) 7.5202
68m 49s (- 0m 0s) (10000 100%) 7.6157


In [41]:
evaluateRandomly(encoder1, attn_decoder1)

> tensor([    0,   316, 31932,     2,   532,  1925, 44354,    39,   479, 16367,
           14,   444,     5,   532,    21,    33,  1236, 52440,     2,   211,
        10310,    51,    24,   330,    14,  8968,   964,     8,   508,  1247,
          477,  1187,   444,     3,     1], device='cuda:0')
= tensor([   0, 4007, 1337,   43, 1247,  477, 1187,    1], device='cuda:0')
< <SOS> : : , , <EOS>

> tensor([     0,  29977,   2169,   1266,      2,    223,    302,      5,    451,
          7290,     12,    777,    582,      8,   1910,    170,     27,     13,
          1781, 131270,     37,   5964,      3,    160,     30,      6,   5272,
           189,      5,   8427,    660,     10,    301,   6085,    486,      3,
             1], device='cuda:0')
= tensor([    0,  2169,   777,    13, 93305,     2, 10098,     8,  5272,   189,
            1], device='cuda:0')
< <SOS> : , , , <EOS>

> tensor([     0,    408,     41,  13346,     72,      2,   1144,    540,     36,
           986,     47,    167

In [42]:
# !pip install rouge

In [43]:
# from rouge import Rouge
# model_out = "he began by starting a five person war cabinet and included chamberlain as lord president of the council"

# reference = "he began his premiership by forming a five-man war cabinet which included chamberlain as lord president of the council"
# rouge = Rouge()
# rouge.get_scores(model_out, reference)